In [2]:
from skimage import io
import os
from my_functions.otsu import otsu_thresholding 
def anwendung(function,input_path,result_path):
    files = os.listdir(input_path)
    for file in files:
        file_path = os.path.join(input_path,file)
        result_file_path = os.path.join(result_path,file)
        result = function(io.imread(file_path))
        io.imsave(result_file_path,result)

In [ ]:
#Otsu Anwendung auf Datensatz 1 und 3

#Anwendung auf ungefilterte Bilder

#N2DH-GOWT1
anwendung(otsu_thresholding,'../BilderDaten/BilderDaten/N2DH-GOWT1/img/','../Ergebnisse/Otsu/')


#NIH3T3
anwendung(otsu_thresholding,'../BilderDaten/BilderDaten/NIH3T3/img/','../Ergebnisse/Otsu/NIH3T3/')
    
#Anwendung auf meanfilter Bilder

#N2DH-GOWT1
anwendung(otsu_thresholding,'../Ergebnisse/Mean/N2DH-GOWT1/','../Ergebnisse/Mean+Otsu/N2DH-GOWT1/')

#NIH3T3
anwendung(otsu_thresholding,'../Ergebnisse/Mean/NIH3T3/','../Ergebnisse/Mean+Otsu/NIH3T3/')

#Anwendung auf medianfilter bilder

#N2DH-GOWT1
anwendung(otsu_thresholding,'../Ergebnisse/Median/N2DH-GOWT1/','../Ergebnisse/Median+Otsu/N2DH-GOWT1/')

#NIH3T3
anwendung(otsu_thresholding,'../Ergebnisse/Median/NIH3T3/','../Ergebnisse/Median+Otsu/NIH3T3/')

#anwendung auf histogramm stretching bilder

#N2DH-GOWT1
anwendung(otsu_thresholding,'../Ergebnisse/Histogramm_stretching/N2DH-GOWT1/','../Ergebnisse/Histogramm+Otsu/N2DH-GOWT1/')

#NIH3T3
anwendung(otsu_thresholding,'../Ergebnisse/Histogramm_stretching/NIH3T3/','../Ergebnisse/Histogramm+Otsu/NIH3T3/')

#anwendung auf gauss bilder

#N2DH-GOWT1
anwendung(otsu_thresholding,'../Ergebnisse/Gauss/N2DH-GOWT1/','../Ergebnisse/Gauss+Otsu/N2DH-GOWT1/')

#NIH3T3
anwendung(otsu_thresholding,'../Ergebnisse/Gauss/NIH3T3/','../Ergebnisse/Gauss+Otsu/NIH3T3/')
‚